In [ ]:
%run AMINonVee.ipynb

In [ ]:
from importlib import reload
#reload(Utilities)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version
import itertools
from dateutil.parser import parse

from pmdarima import auto_arima
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import month_plot, quarter_plot, seasonal_plot
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX

from arch import arch_model

import pyodbc
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
import Plot_Box_sns
import GrubbsTest
import DickeyFullerTest as dft

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
fig_num = 0

In [ ]:
cols_of_interest = [
    'serialnumber',
    'prem_nb', 
    'srvc_pole_nb', 
    'trsf_pole_nb',
    'annual_kwh',
    'station_nb',
    'xfmr_nb', 
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset', 
    'aep_derived_uom',
    'aep_srvc_qlty_idntfr', 
    'value', 
    'aep_usage_dt'
]

In [ ]:
df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full_NEW\outg_rec_nb_11496239')
#df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full\outg_rec_nb_11751094')
out_t_beg_local = pd.to_datetime('2020-03-05 11:03:00')
out_t_end_local = pd.to_datetime('2020-03-05 12:40:00')

In [ ]:
# df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11770899')
# out_t_beg = pd.to_datetime('2020-07-01 14:18:00')
# out_t_end = pd.to_datetime('2020-07-01 18:50:00')

In [ ]:
kwh_vlt_dfs_dict = AMINonVee.assemble_kwh_vlt_dfs_from_saved_csvs(file_dir=df_outage_dir, glob_pattern=r'outg_rec_nb_*.csv', 
                                                                  cols_of_interest=cols_of_interest, 
                                                                  verbose=True)
df_kwh_15T = kwh_vlt_dfs_dict['kwh']
df_vlt_15T = kwh_vlt_dfs_dict['vlt']

In [ ]:
df_vlt_15T.shape

In [ ]:
df_kwh_15T.shape

In [ ]:
df_kwh_15T['aep_srvc_qlty_idntfr'].value_counts()

In [ ]:
df_vlt_15T['aep_srvc_qlty_idntfr'].value_counts()

In [ ]:
df_kwh_15T.head()

In [ ]:
df_key = 'df'
df_agg_key = 'df_agg'

In [ ]:
freqs=['H', '4H', 'D', 'MS']
time_col_for_agg='endtimeperiod_utc'
other_grouper_cols=['serialnumber']
build_agg_dfs=True
other_cols_to_keep = ['prem_nb', 'srvc_pole_nb', 'trsf_pole_nb', 'station_nb', 
                      'xfmr_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']
other_cols_to_keep_agg = ['aep_derived_uom', 'aep_srvc_qlty_idntfr']
agg_cols=['value', 'annual_kwh']
agg_types=['mean']
mix_agg_functions=False

#-------------------------------------------------------------------------------------------------
resampled_dfs_kwh_dict = AMINonVee.build_time_resampled_dfs(df_kwh_15T, base_freq='15T', freqs=freqs, 
                                           other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                           build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                           agg_cols=agg_cols, agg_types=agg_types, 
                                           other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                           df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_kwh_15T.equals(resampled_dfs_kwh_dict['15T'][df_key]))
df_kwh_15T_mean = resampled_dfs_kwh_dict['15T'][df_agg_key]
#-----
df_kwh_H = resampled_dfs_kwh_dict['H'][df_key]
df_kwh_H_mean = resampled_dfs_kwh_dict['H'][df_agg_key]
#-----
df_kwh_4H = resampled_dfs_kwh_dict['4H'][df_key]
df_kwh_4H_mean = resampled_dfs_kwh_dict['4H'][df_agg_key]
#-----
df_kwh_D = resampled_dfs_kwh_dict['D'][df_key]
df_kwh_D_mean = resampled_dfs_kwh_dict['D'][df_agg_key]
#-----
df_kwh_M = resampled_dfs_kwh_dict['MS'][df_key]
df_kwh_M_mean = resampled_dfs_kwh_dict['MS'][df_agg_key]
#-------------------------------------------------------------------------------------------------
resampled_dfs_vlt_dict = AMINonVee.build_time_resampled_dfs(df_vlt_15T, base_freq='15T', freqs=freqs, 
                                           other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                           build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                           agg_cols=agg_cols, agg_types=agg_types, 
                                           other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                           df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_vlt_15T.equals(resampled_dfs_vlt_dict['15T'][df_key]))
df_vlt_15T_mean = resampled_dfs_vlt_dict['15T'][df_agg_key]
#-----
df_vlt_H = resampled_dfs_vlt_dict['H'][df_key]
df_vlt_H_mean = resampled_dfs_vlt_dict['H'][df_agg_key]
#-----
df_vlt_4H = resampled_dfs_vlt_dict['4H'][df_key]
df_vlt_4H_mean = resampled_dfs_vlt_dict['4H'][df_agg_key]
#-----
df_vlt_D = resampled_dfs_vlt_dict['D'][df_key]
df_vlt_D_mean = resampled_dfs_vlt_dict['D'][df_agg_key]
#-----
df_vlt_M = resampled_dfs_vlt_dict['MS'][df_key]
df_vlt_M_mean = resampled_dfs_vlt_dict['MS'][df_agg_key]
#-----

# Convert out_t_beg_local and out_t_end_local to UTC

In [ ]:
[out_t_beg, out_t_end] = Utilities_dt.determine_timezone_and_convert_local_to_utc_time([out_t_beg_local, out_t_end_local], 
                                                                                       pd.to_timedelta(df_kwh_15T['timezoneoffset'].unique()).tolist())

# 15-minute granularity

In [ ]:
expand_time = datetime.timedelta(hours=4)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
df_kwh_15T_mean.columns

In [ ]:
expand_time = datetime.timedelta(hours=4)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'))

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'))
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(hours=24)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_15T_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'))

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'))
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=14)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'))

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_15T_mean, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'15-min Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'))
fig_num +=1

# Hour granularity

In [ ]:
expand_time = datetime.timedelta(hours=24)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=7)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
df_kwh_4H.columns

# 4-Hour granularity

In [ ]:
expand_time = datetime.timedelta(days=7)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_4H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'4-Hour Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_4H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_4H, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'4-Hour Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_4H_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

# Daily granularity

In [ ]:
expand_time = datetime.timedelta(days=50)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)


fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_D, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Daily Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_D_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_D, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Daily Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_D_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.5, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=50)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)


fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_D, 
                                  x='date', y='mean_TRS value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Daily Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_D_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_D, 
                                  x='date', y='mean_TRS value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Daily Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_D_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.5, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

# Month granularity

In [ ]:
expand_time = datetime.timedelta(days=365*10)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)


fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_M, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_M_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_M, 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_M_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.5, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=365*10)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)


fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_M, 
                                  x='date', y='mean_TRS value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_kwh_M_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_vlt_M, 
                                  x='date', y='mean_TRS value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=df_vlt_M_mean, df_mean_col='value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.5, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True)
fig_num +=1